# import data

In [1]:
import platform
import pandas as pd
from datetime import datetime, timedelta, time
ohlcv_data = pd.read_pickle('/Users/bokie/code/python/CMS/可转债套利/cms/stock_data_min_new.pkl')
print(ohlcv_data.keys())
for k,v in ohlcv_data.items():
    print(k,' data length: ',len(v))
print('数据起始日期',v.index[0])
print('数据结束日期',v.index[-1])
print('113642.SH数据起始日期',ohlcv_data['113642.SH'].index[0])
print('113642.SH数据结束日期',ohlcv_data['113642.SH'].index[-1])

dict_keys(['123046.SZ', '300587.SZ', '113642.SH', '603185.SH', '110061.SH', '600674.SH', '128081.SZ', '002203.SZ'])
123046.SZ  data length:  31460
300587.SZ  data length:  31460
113642.SH  data length:  13552
603185.SH  data length:  31460
110061.SH  data length:  31460
600674.SH  data length:  31460
128081.SZ  data length:  31460
002203.SZ  data length:  31460
数据起始日期 2021-12-13 09:30:00
数据结束日期 2022-06-28 15:00:00
113642.SH数据起始日期 2022-04-06 09:30:00
113642.SH数据结束日期 2022-06-28 15:00:00


# bond-stock pair

In [2]:
# cbond_pair = ['128081.SZ','002203.SZ']
# part = 1
# convert_price = 9.69
# stock_volume = 100/convert_price #整个回测期间不变


cbond_pair = ['123046.SZ','300587.SZ']
part = 1 
convert_price = 6.73
stock_volume = 100/convert_price  # 2021-12-13至2022-03-23
# part = 2
# convert_price = 6.74
# stock_volume = 100/convert_price  # 2022-03-24至2022-06-28


# cbond_pair = ['110061.SH','600674.SH']
# part = 1
# convert_price = 9.2
# stock_volume = 100/convert_price #整个回测期间不变


# cbond_pair = ['113642.SH','603185.SH']
# part = 1
# convert_price = 145.66
# stock_volume = 100/convert_price  # 2022-04-06至2022-06-02
# part = 2
# convert_price = 102.61
# stock_volume = 100/convert_price  # 2022-06-03至2022-06-28


In [3]:
cbond_pair_price = pd.DataFrame([])
cbond_pair_price = pd.concat([ohlcv_data[cbond_pair[0]]['close'],ohlcv_data[cbond_pair[1]]['close']],axis=1)
cbond_pair_price.columns = cbond_pair
cbond_pair_price = cbond_pair_price.dropna()
if cbond_pair[0] == '128081.SZ' or cbond_pair[0] == '110061.SH':
    cbond_pair_price = cbond_pair_price.loc['2021-12-13 15:00:00':]
elif cbond_pair[0] == '123046.SZ' and part == 1:
    cbond_pair_price = cbond_pair_price.loc['2021-12-13 15:00:00':'2022-03-23 15:00:00']
elif cbond_pair[0] == '123046.SZ' and part == 2:
    cbond_pair_price = cbond_pair_price.loc['2022-03-25 15:00:00':]
elif cbond_pair[0] == '113642.SH' and part == 1:
    cbond_pair_price = cbond_pair_price.loc['2022-04-07 15:00:00':'2022-06-02 15:00:00']
elif cbond_pair[0] == '113642.SH' and part == 2:
    cbond_pair_price = cbond_pair_price.loc['2022-06-03 15:00:00':]


# signal calculate

In [4]:
cbond_return = pd.DataFrame([])

cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_pair_price.iloc[0,1] # 初始时刻的stock前收盘价

for index,row in cbond_pair_price.iterrows():
    
    cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    cbond_return.loc[index,cbond_pair[1]] = row[cbond_pair[1]]/stock_preclose - 1

    if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
        cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
        stock_preclose = row[cbond_pair[1]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

0

In [5]:
pd.set_option('display.max_rows', None)
# pair moving trend: diff = cbond reutrn - stock return 
cbond_return['diff'] = cbond_return[cbond_pair[0]] - cbond_return[cbond_pair[1]]

len(cbond_return)

15107

# grid search

In [6]:
import warnings
warnings.filterwarnings("ignore")
grid_search_res = {}
signal_res = {}
trade_res = {}

In [7]:
open_threshold_list = [0.005,0.006,0.007,0.008,0.009,
                        0.01,0.011,0.012,0.013,0.014,0.015,
                        # 0.016,0.017,0.018,0.019,0.020,0.021,0.022 # 0.005 ~ 0.01 ~ 0.015 间隔是0.001
                        ]

close_threshold_list = [0,0.001,0.002,0.003] #  0 ~ 0.001 ~ 0.002 ~ 0.003
# close_threshold_list = [0.004,0.0045] #  0 ~ 0.001 ~ 0.002 ~ 0.003
for open_threshold in open_threshold_list[:]:
    for close_threshold in close_threshold_list[:]:
        ######################################################################################################################################################
        # open_threshold = 0.01  # 0.005 ~ 0.01 ~ 0.015 间隔是0.001
        # open_threshold = 0.008
        # close_threshold = 0.001 #  0 ~ 0.001 ~ 0.002 ~ 0.003
        pos_open_flag = 0
        neg_open_flag = 0
        cbond_position = pd.DataFrame([])
        cbond_position.index = cbond_return.index
        for index,row in cbond_return.iterrows():

            if row['diff'] >= open_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff落在A区间，且无仓位，正开
                pos_open_flag = 1
                neg_open_flag = 0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] >= open_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff落在A区间，且有负仓，平仓再正开
                pos_open_flag=1
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] >= open_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff落在A区间，且有正仓，keep
                pos_open_flag=1
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag

            if row['diff'] > close_threshold and row['diff'] < open_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在B区间，且有正仓，keep
                pos_open_flag=1
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] > close_threshold and row['diff'] < open_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在B区间，且有负仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            elif row['diff'] > close_threshold and row['diff'] < open_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在B区间，且无仓，keep
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            
            if row['diff'] >= -1*close_threshold and row['diff'] <= close_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在C区间，且有正仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] >= -1*close_threshold and row['diff'] <= close_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在C区间，且有负仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            elif row['diff'] >= -1*close_threshold and row['diff'] <= close_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在C区间，且无仓位，keep
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            
            if row['diff'] > -1*open_threshold and row['diff'] < -1*close_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在D区间，且有负仓，keep
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag

            elif row['diff'] > -1*open_threshold and row['diff'] < -1*close_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在D区间，且有正仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] > -1*open_threshold and row['diff'] < -1*close_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在D区间，且无仓位，keep
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag

            if row['diff'] <= -1*open_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在E区间，且无仓位，反开
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag
            elif row['diff'] <= -1*open_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在E区间，且有正仓，平仓反开
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag
            elif row['diff'] <= -1*open_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在E区间，且有负仓，keep
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag

            if index.time()==time(15,0,0): # 每日收盘时必须平仓（时间止损信号）
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
        ######################################################################################################################################################
        # cbond_position.to_csv('cbond_position.csv')
        df_signal = pd.concat([cbond_pair_price,cbond_return,cbond_position],axis=1)
        df_signal.columns = ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
        # df_signal.to_csv('df_signal.csv')

        # 用来记录交易流水的表格,一边遍历df_signal，一边填入内容到流水表格里

        import numpy as np
        cols = ['balance','signal','bond price','stock price','bond hold','stock hold','commission','tax','reverse open']
        df_trade = pd.DataFrame(
            data = [[0,np.nan,np.nan,np.nan,0,0,0,0,np.nan]],
            columns=cols
            ) 

        balance_last = 0 # 现金账户初始余额

        bond_hold = 0 # 债券持有数量
        stock_hold = 0 # 股票持有数量

        commission_rate = 2/10000 # 佣金买卖万二
        tax_rate = 1/1000 #



        for i in range(len(df_signal)-1):
            if df_signal.values[i][-1] != df_signal.values[i+1][-1]:
                index = df_signal.index[i+1]

            # df_signal的columns
            # ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
                signal = str(int(df_signal.values[i+1][-2]))+str(int(df_signal.values[i+1][-1]))
                
                bond_price = df_signal.values[i+1][0] 

                stock_price = df_signal.values[i+1][1]

                commission = (bond_price + stock_price * stock_volume) * commission_rate

                reverse_open = False #表明这笔交易，是否包含平仓后再反向开仓的操作

                if signal == '00' and stock_hold == stock_volume: #有股票持仓并且需要卖出平仓时，有印花税
                    tax = stock_price * tax_rate * stock_volume
                else:
                    tax = 0

                if signal == '00':
                    balance_last += bond_hold*bond_price
                    balance_last += stock_hold*stock_price
                    
                    bond_hold = 0
                    stock_hold = 0
                    

                if signal == '1-1':
                    '''
                    加一个检测之前持仓的模块,如果之前因为-11的信号有持仓,需要先平掉(即买债卖股票),同时再反向开仓
                    '''
                    reverse_flag = False
                    if bond_hold == -1 and stock_hold == 1*stock_volume:
                        bond_hold = 0
                        stock_hold = 0
                        balance_last -= bond_price 
                        balance_last += stock_price * stock_volume
                        commission *= 2 # 因为平掉之后再反向开仓，手续费double
                        tax = stock_price * tax_rate * stock_volume # 卖股票要收印花税
                        reverse_flag = True
                        reverse_open = True

                    bond_hold += 1
                    stock_hold += (-1 * stock_volume)
                    # stock_hold += stock_volume
                    balance_last -= bond_price
                    balance_last += stock_price * stock_volume
                    
                    if reverse_flag:
                        tax *= 2 #平仓然后反向开仓的时候 已经计算过tax了，直接double
                    else:
                        tax = stock_price * tax_rate * stock_volume # 卖股票要收印花税

                if signal == '-11':
                    '''
                    加一个检测之前持仓的模块,如果之前因为1-1的信号有持仓,需要先平掉(卖债买股票),同时再反向开仓
                    '''
                    if bond_hold == 1 and stock_hold == -1*stock_volume:
                        bond_hold = 0
                        stock_hold = 0
                        balance_last += bond_price
                        balance_last -= stock_price * stock_volume
                        commission *= 2 # 因为平掉之后再反向开仓，手续费double
                                        # 买股票无印花税
                        reverse_open = True
                    bond_hold += -1
                    stock_hold += (1 * stock_volume)
                    # stock_hold += stock_volume
                    balance_last += bond_price
                    balance_last -= stock_price * stock_volume
                

                balance = balance_last

                line = [balance,signal,bond_price,stock_price,bond_hold,stock_hold,commission,tax,reverse_open]
                # df_trade.loc[index,['balance','asset','signal','bond price','stock price','commission','tax']] = line
                # df_trade.loc[index,cols] = [0,0,np.nan,np.nan,np.nan,0,0,0,0]
                df_trade.loc[index,cols] = line

        df_trade['net value'] = df_trade['balance'] + \
                                df_trade['bond price']*df_trade['bond hold'] + \
                                df_trade['stock price']*df_trade['stock hold'] - \
                                df_trade['commission'] - \
                                df_trade['tax']       
        #把交易序列单个时间节点的扩展到整个序列
        # assert False
        df_nv = df_trade[['net value']].iloc[1:]
        df_nv['time'] = df_nv.index
        df_tmp = df_signal[['diff']]
        df_tmp['time'] = df_tmp.index
        #  merge之后再前向往后fillna
        df_nv_full =  pd.merge(df_nv,df_tmp,how='outer').sort_values(by='time').fillna(method='ffill').fillna(0)
        df_nv_full.index = df_tmp.index
        df_nv_full = df_nv_full[['net value']]
        df_trade_2 = df_trade.copy(deep=True)
        rounds = [] #只包含单个来回，每2条交易记录组成一个来回的组
        reverse_rounds = [] # 包含多个来回，信号之间包含平仓同时反向开仓的组
        for i in range(len(df_trade)-1,0,-1):
        # for i in range(len(df_trade)-1,-1,-1):
            if df_trade_2.iloc[i-1,:]['signal'] == '00':
                round = df_trade_2.iloc[i:,:]
                if len(round)>2:
                    reverse_rounds.append(round)
                else:
                    rounds.append(round)
                df_trade_2 = df_trade_2.iloc[:i]

        num_round = len(rounds)
        for i in reverse_rounds:
            num_round += len(i)-1
        # # print('single round:',num_round)
        # # print('reverse rounds: ',len(reverse_rounds))

        return_in_rounds = []
        win = 0
        lose = 0
        win_profit = 0
        lose_profit = 0
        for i in rounds:
            
            tmp_profit = (i['net value'] - i['net value'].shift(1)).values[-1]
            single_round_cost = (i['bond price'].sum() + (i['stock price']*stock_volume).sum())/4
            
            return_in_round = tmp_profit/single_round_cost
            return_in_rounds.append(return_in_round)
            
            
            if tmp_profit>0:
                win += 1
                win_profit += tmp_profit
            else:
                lose += 1
                lose_profit += tmp_profit
        # # print('win: ',win)
        # # print('lose: ',lose)
        # # print('win profit: ',win_profit)
        # # print('lose profit: ',lose_profit)
        # # print(len(return_in_rounds))

        ''' 
        先把连续反向开仓的信号拆开
        e.g.  1-1 -11 00 ---> 1-1 00 -11 00
        '''

        reverse_rounds_reconstruct = []
        for reverse_round in reverse_rounds:
            reverse_rounds0_reconstruct = []
            for i in range(0,len(reverse_round)-1): #4-2

                # tmp_cols = reverse_rounds[0].columns
                # tmp_index = reverse_rounds[0].iloc[i,:].reset_index()
                m = reverse_round.iloc[i,:].tolist()
                n = reverse_round.iloc[i+1,:].tolist()
                # reverse_rounds0_reconstruct.append(m)
                tmp_balance = m[0] + n[2]*m[4] + n[3]*m[5] 
                tmp_commission = n[6]/2
                tmp_tax = n[7]/2
                tmp_nv =  tmp_balance - tmp_commission - tmp_tax
                
                insert_line  = [tmp_balance,'00',np.nan,np.nan,0,0,tmp_commission,tmp_tax,'-',tmp_nv]
                reverse_rounds0_reconstruct.append(m)
                reverse_rounds0_reconstruct.append(insert_line)
            # reverse_rounds0_reconstruct.append(n)    
            tmp_cols = reverse_round.columns
            reverse_rounds0_reconstruct[-1] = reverse_round.iloc[-1,:].tolist()
            tmp_round = pd.DataFrame(reverse_rounds0_reconstruct[:],columns=tmp_cols)
            tmp_round
            for index,row in tmp_round.iterrows():
                if row['reverse open'] == True:
                    tmp_round.loc[index,['commission','tax']] = tmp_round.loc[index,['commission','tax']]/2
            tmp_round = tmp_round.fillna(method = 'bfill')
            
            tmp_round['new nv'] = tmp_round['balance'] + tmp_round['bond price']*tmp_round['bond hold'] + tmp_round['stock price']*tmp_round['stock hold'] - tmp_round['commission'] - tmp_round['tax']
            reverse_rounds_reconstruct.append(tmp_round)

        # reverse_rounds_reconstruct[0]
        reverse_rounds_detail = []
        for reverse_round in reverse_rounds_reconstruct:
            # print(len(reverse_round))
            for i in range(0,len(reverse_round)-1,2):
                df_tmp = reverse_round.iloc[i:i+2,:]
                reverse_rounds_detail.append(df_tmp)

        # print(len(reverse_rounds_detail))

        # return_in_rounds = []
        # win = 0
        # lose = 0
        # win_profit = 0
        # lose_profit = 0
        for i in reverse_rounds_detail:
            
            tmp_profit = (i['new nv'] - i['new nv'].shift(1)).values[-1]
            single_round_cost = (i['bond price'].sum() + (i['stock price']*stock_volume).sum())/4
            
            return_in_round = tmp_profit/single_round_cost
            return_in_rounds.append(return_in_round)
            
            
            if tmp_profit>0:
                win += 1
                win_profit += tmp_profit
            else:
                lose += 1
                lose_profit += tmp_profit
        # print('win: ',win)
        # print('lose: ',lose)
        # print('win profit: ',win_profit)
        # print('lose profit: ',lose_profit)
        # print(len(return_in_rounds))
        ###################################################################################################
        close_by_time = 0
        close_by_time_win = 0
        close_by_time_lose = 0

        close_by_signal = 0
        close_by_signal_win = 0
        close_by_signal_lose = 0

        for tmp_round in rounds:
            tmp_profit = tmp_round['net value'].tolist()[1] - tmp_round['net value'].tolist()[0]
            if tmp_round.index[1].time() == time(15,0,0):
                close_by_time += 1
                if tmp_profit > 0:
                    close_by_time_win += 1
                else:
                    close_by_time_lose += 1
            else:
                close_by_signal += 1
                if tmp_profit > 0:
                    close_by_signal_win += 1
                else:
                    close_by_signal_lose += 1

        #############################################################
        #还有bug未解决
        for i in range(len(reverse_rounds)-1):
            tmp_round = reverse_rounds[i]
            tmp_round_reconstruct = reverse_rounds_reconstruct[i]
            print(tmp_round)
            if tmp_round.index[-1].time() == time(15,0,0):
                close_by_time += 1
                if tmp_profit > 0:
                    close_by_time_win += 1
                else:
                    close_by_time_lose += 1
            new_nv =  tmp_round_reconstruct['new nv'].tolist()[:-2] #最后一组round丢掉
            for i in range(1,len(new_nv)-3,2):      # 最后一组round丢掉，所以减3而不是减1            
                tmp_profit = new_nv[i] - new_nv[i-1]
                close_by_signal += 1
                if tmp_profit > 0:
                    close_by_signal_win += 1
                else:
                    close_by_signal_lose += 1
        #############################################################                    
        # print('close_by_time: ',close_by_time)
        # print('close_by_time_win: ',close_by_time_win)
        # print('close_by_time_lose: ',close_by_time_lose)
        # print()
        # print('close_by_signal: ',close_by_signal)
        # print('close_by_signal_win: ',close_by_signal_win)
        # print('close_by_signal_lose: ',close_by_signal_lose)
        ###################################################################################################
        win_rate = win/(win+lose)
        win_profit_avg = win_profit/win
        lose_profit_avg = lose_profit/lose
        profit_ratio = win_profit_avg/abs(lose_profit_avg)

        total_round = win + lose 

        trade_days = len(df_nv_full.resample('D').last().dropna()) -1 
        round_per_day = total_round/trade_days
        return_per_round = sum(return_in_rounds)/len(return_in_rounds)
        # print('$'*50)
        # print('pair code:',cbond_pair)
        # print('convert price: ',convert_price)
        # print('first trade time',df_trade.index[1])
        # print('last trade time',df_trade.index[-1])
        # print('$'*50)
        # print('win: ',win)
        # print('lose: ',lose)
        # print('win profit: ',win_profit)
        # print('lose profit: ',lose_profit)
        # print('sum of win&lose profit: ',win_profit+lose_profit)
        # print('final net value:',df_trade['net value'].values[-1])
        # print('win rate: ', win_rate)
        # print('profit ratio: ',profit_ratio)
        # print('trade days: ',trade_days)
        # print('total round: ',total_round)
        # print('avg round in a day: ',round_per_day)
        # print('return_per_round: ',return_per_round)
        # print('@'*80)
        df_res = pd.DataFrame(data = [[np.nan]*20],
            columns=['cbond pair','convert price','win','lose','win profit','lose profit',
            'sum of win&lose profit','final net value','win rate',
            'profit ratio','trade days','total round','avg round in a day','return per round',
            'close by time','close by time win','close by time lose',
            'close by signal','close by signal win','close by signal lose'])

        tmp_idx = str(df_trade.index[1]) + ' -> ' + str(df_trade.index[-1])
        tmp_line = [cbond_pair,convert_price,win,lose,win_profit,lose_profit,
                    win_profit+lose_profit,df_trade['net value'].values[-1],win_rate,
                    profit_ratio,trade_days,total_round,round_per_day,return_per_round,
                    close_by_time,close_by_time_win,close_by_time_lose,
                    close_by_signal,close_by_signal_win,close_by_signal_lose]

        df_res.loc[tmp_idx,df_res.columns] = tmp_line
        df_res = df_res.dropna()
        df_res['open threshold'] = open_threshold
        df_res['close threshold'] = close_threshold
        
        tmp_key = str(open_threshold) +'@'+ str(close_threshold)
        grid_search_res[tmp_key] = df_res
        signal_res[tmp_key] = df_signal 
        trade_res[tmp_key] = df_trade
        print(tmp_key)

0.005@0
0.005@0.001
0.005@0.002
0.005@0.003
0.006@0
0.006@0.001
0.006@0.002
0.006@0.003
0.007@0
0.007@0.001
0.007@0.002
0.007@0.003
0.008@0
0.008@0.001
0.008@0.002
0.008@0.003
0.009@0
0.009@0.001
0.009@0.002
0.009@0.003
0.01@0
0.01@0.001
0.01@0.002
0.01@0.003
0.011@0
0.011@0.001
0.011@0.002
0.011@0.003
0.012@0
0.012@0.001
0.012@0.002
0.012@0.003
0.013@0
0.013@0.001
0.013@0.002
0.013@0.003
0.014@0
0.014@0.001
0.014@0.002
0.014@0.003
0.015@0
0.015@0.001
0.015@0.002
0.015@0.003


In [8]:
# print(len(rounds))
# print(len(df_trade))
# rounds[0]

In [9]:
# df_trade

In [10]:
df_grid_search_res =  pd.concat(list(grid_search_res.values()),axis=0)

In [11]:
df_grid_search_res.columns

Index(['cbond pair', 'convert price', 'win', 'lose', 'win profit',
       'lose profit', 'sum of win&lose profit', 'final net value', 'win rate',
       'profit ratio', 'trade days', 'total round', 'avg round in a day',
       'return per round', 'close by time', 'close by time win',
       'close by time lose', 'close by signal', 'close by signal win',
       'close by signal lose', 'open threshold', 'close threshold'],
      dtype='object')

In [12]:
df_grid_search_res[['win','lose','total round',
                    'close by time','close by signal',
                    'close by time win','close by time lose',
                    'close by signal win','close by signal lose',
                    'open threshold', 'close threshold'
                    ]]

,win,lose,total round,close by time,close by signal,close by time win,close by time lose,close by signal win,close by signal lose,open threshold,close threshold
2021-12-14 09:30:00 -> 2022-03-23 15:00:00,183.0,41.0,224.0,56.0,166.0,16.0,40.0,166.0,0.0,0.005,0.000
2021-12-14 09:30:00 -> 2022-03-23 15:00:00,212.0,41.0,253.0,55.0,196.0,16.0,39.0,195.0,1.0,0.005,0.001
2021-12-14 09:30:00 -> 2022-03-23 15:00:00,268.0,41.0,309.0,55.0,252.0,16.0,39.0,251.0,1.0,0.005,0.002
2021-12-14 09:30:00 -> 2022-03-23 15:00:00,334.0,43.0,377.0,53.0,322.0,13.0,40.0,320.0,2.0,0.005,0.003
2021-12-14 09:32:00 -> 2022-03-23 15:00:00,149.0,39.0,188.0,55.0,131.0,17.0,38.0,131.0,0.0,0.006,0.000
2021-12-14 09:32:00 -> 2022-03-23 15:00:00,166.0,40.0,206.0,53.0,151.0,15.0,38.0,150.0,1.0,0.006,0.001
2021-12-14 09:32:00 -> 2022-03-23 15:00:00,199.0,40.0,239.0,53.0,184.0,15.0,38.0,183.0,1.0,0.006,0.002
2021-12-14 09:32:00 -> 2022-03-23 15:00:00,239.0,39.0,278.0,50.0,226.0,13.0,37.0,225.0,1.0,0.006,0.003
2021-12-14 10:33:00 -> 2022-03-23 15:00:00,109.0,38.0,147.0,51.0,94.0,14.0,37.0,94.0,0.0,0.007,0.000
2021-12-14 10:33:00 -> 2022-03-23 15:00:00,121.0,39.0,160.0,49.0,109.0,12.0,37.0,108.0,1.0,0.007,0.001


In [60]:
trade_res['0.005@0.001']

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
0,0.000000,NaN,NaN,NaN,0.0,0.000000,0.000000,0.000000,NaN,NaN
2021-12-14 09:30:00,54.710122,-11,351.144,19.95,-1.0,14.858841,0.129516,0.000000,False,-0.129516
2021-12-14 09:34:00,-47.733061,1-1,348.547,20.01,1.0,-14.858841,0.258349,0.594651,True,2.635531
2021-12-14 09:38:00,6.072942,00,351.280,20.02,0.0,0.000000,0.129751,0.000000,False,5.943191
2021-12-14 09:43:00,-44.913826,1-1,348.015,19.99,1.0,-14.858841,0.129009,0.297028,False,5.646905
2021-12-14 15:00:00,4.341003,00,344.500,19.87,0.0,0.000000,0.127949,0.000000,False,4.213054
2021-12-15 09:30:00,-43.317530,1-1,344.241,19.96,1.0,-14.858841,0.128165,0.296582,False,3.916256
2021-12-15 13:07:00,5.385536,00,341.868,19.73,0.0,0.000000,0.127007,0.000000,False,5.258530
2021-12-15 13:50:00,55.453079,-11,339.072,19.45,-1.0,14.858841,0.125615,0.000000,False,5.259921
2021-12-15 14:13:00,6.854478,00,338.346,19.50,0.0,0.000000,0.125619,0.289747,False,6.439112


# return per round surface

In [13]:
df_return_per_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_return_per_round.loc[open_threshold,close_threshold] = row['return per round']
print('max return per round: \n',df_return_per_round.max())
print('min return per round: \n',df_return_per_round.min())
df_return_per_round

max return per round: 
 0.000    0.006380
0.001    0.006139
0.002    0.006217
0.003    0.006270
dtype: float64
min return per round: 
 0.000    0.004226
0.001    0.003919
0.002    0.003496
0.003    0.003101
dtype: float64


,0.000,0.001,0.002,0.003
0.005,0.004226,0.003919,0.003496,0.003101
0.006,0.004465,0.004182,0.003773,0.003457
0.007,0.004483,0.004274,0.003843,0.003584
0.008,0.004695,0.004473,0.004146,0.003956
0.009,0.004866,0.004698,0.004324,0.004275
0.010,0.005150,0.004817,0.004496,0.004362
0.011,0.005208,0.004833,0.004491,0.004452
0.012,0.005763,0.005342,0.005179,0.005086
0.013,0.006232,0.005867,0.005756,0.005683
0.014,0.006380,0.006139,0.006217,0.006270


In [14]:
import plotly.graph_objects as go

x_data = df_return_per_round.columns.to_list() # close threshold

y_data = df_return_per_round.index.to_list() # open threshold

z_data = df_return_per_round.copy(deep=True) # return per round

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="return per round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# win rate surface

In [15]:
df_win_rate = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_win_rate.loc[open_threshold,close_threshold] = row['win rate']
print('max win rate: \n',df_win_rate.max())
print('min win rate: \n',df_win_rate.min())    
df_win_rate

max win rate: 
 0.000    0.816964
0.001    0.837945
0.002    0.867314
0.003    0.885942
dtype: float64
min win rate: 
 0.000    0.655738
0.001    0.661290
0.002    0.677419
0.003    0.693878
dtype: float64


,0.000,0.001,0.002,0.003
0.005,0.816964,0.837945,0.867314,0.885942
0.006,0.792553,0.805825,0.832636,0.859712
0.007,0.741497,0.756250,0.782857,0.808081
0.008,0.706349,0.725926,0.753521,0.778481
0.009,0.709091,0.728814,0.739837,0.772059
0.010,0.683168,0.692308,0.703704,0.728070
0.011,0.666667,0.674157,0.677419,0.693878
0.012,0.657895,0.666667,0.679012,0.694118
0.013,0.666667,0.671429,0.698630,0.710526
0.014,0.655738,0.661290,0.707692,0.720588


In [16]:
import plotly.graph_objects as go

x_data = df_win_rate.columns.to_list() # close threshold

y_data = df_win_rate.index.to_list() # open threshold

z_data = df_win_rate.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="win rate", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# total round surface

In [17]:
df_total_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_total_round.loc[open_threshold,close_threshold] = row['total round']
print('max total round: \n',df_total_round.max())
print('min total round: \n',df_total_round.min())    
df_total_round

max total round: 
 0.000    224.0
0.001    253.0
0.002    309.0
0.003    377.0
dtype: float64
min total round: 
 0.000    56.0
0.001    56.0
0.002    58.0
0.003    61.0
dtype: float64


,0.000,0.001,0.002,0.003
0.005,224.0,253.0,309.0,377.0
0.006,188.0,206.0,239.0,278.0
0.007,147.0,160.0,175.0,198.0
0.008,126.0,135.0,142.0,158.0
0.009,110.0,118.0,123.0,136.0
0.010,101.0,104.0,108.0,114.0
0.011,87.0,89.0,93.0,98.0
0.012,76.0,78.0,81.0,85.0
0.013,69.0,70.0,73.0,76.0
0.014,61.0,62.0,65.0,68.0


In [18]:
import plotly.graph_objects as go

x_data = df_total_round.columns.to_list() # close threshold

y_data = df_total_round.index.to_list() # open threshold

z_data = df_total_round.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="total round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# win round surface

In [19]:
df_win_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_win_round.loc[open_threshold,close_threshold] = row['win']
print('max win round: \n',df_win_round.max())
print('min win round: \n',df_win_round.min())    
df_win_round

max win round: 
 0.000    183.0
0.001    212.0
0.002    268.0
0.003    334.0
dtype: float64
min win round: 
 0.000    38.0
0.001    38.0
0.002    41.0
0.003    44.0
dtype: float64


,0.000,0.001,0.002,0.003
0.005,183.0,212.0,268.0,334.0
0.006,149.0,166.0,199.0,239.0
0.007,109.0,121.0,137.0,160.0
0.008,89.0,98.0,107.0,123.0
0.009,78.0,86.0,91.0,105.0
0.010,69.0,72.0,76.0,83.0
0.011,58.0,60.0,63.0,68.0
0.012,50.0,52.0,55.0,59.0
0.013,46.0,47.0,51.0,54.0
0.014,40.0,41.0,46.0,49.0


In [20]:
import plotly.graph_objects as go

x_data = df_win_round.columns.to_list() # close threshold

y_data = df_win_round.index.to_list() # open threshold

z_data = df_win_round.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="win round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# lose round surface

In [21]:
df_lose_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_lose_round.loc[open_threshold,close_threshold] = row['lose']
print('max win round: \n',df_lose_round.max())
print('min win round: \n',df_lose_round.min())    
df_lose_round

max win round: 
 0.000    41.0
0.001    41.0
0.002    41.0
0.003    43.0
dtype: float64
min win round: 
 0.000    18.0
0.001    18.0
0.002    17.0
0.003    17.0
dtype: float64


,0.000,0.001,0.002,0.003
0.005,41.0,41.0,41.0,43.0
0.006,39.0,40.0,40.0,39.0
0.007,38.0,39.0,38.0,38.0
0.008,37.0,37.0,35.0,35.0
0.009,32.0,32.0,32.0,31.0
0.010,32.0,32.0,32.0,31.0
0.011,29.0,29.0,30.0,30.0
0.012,26.0,26.0,26.0,26.0
0.013,23.0,23.0,22.0,22.0
0.014,21.0,21.0,19.0,19.0


In [22]:
import plotly.graph_objects as go

x_data = df_lose_round.columns.to_list() # close threshold

y_data = df_lose_round.index.to_list() # open threshold

z_data = df_lose_round.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="lose round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                )

fig.show()

# signal check

In [23]:
print('open threshold: ',open_threshold,'close threshold: ',close_threshold)
# df_signal

open threshold:  0.015 close threshold:  0.003


In [24]:
len(rounds)

61

In [25]:
cnt = 0
for index, row in df_trade.iterrows():
    
    if str(index)[-8:] == '15:00:00':
        cnt += 1
print(cnt)

34


In [26]:
df_pos_list = []
for k,v in signal_res.items():
    # print(k.split('@'))
    tmp_df_pos = v[['bond_pos','stock_pos']]
    # print(tmp_df_pos.head())
    tmp_df_pos.columns = ['bond_pos'+k,'stock_pos']
    df_pos_list.append(tmp_df_pos)


In [27]:
df_pos = pd.concat([df_signal.iloc[:,:-2],df_pos_list[0],df_pos_list[-4]],axis=1)
print(len(df_pos))
df_pos.head(300)

15107


,bond_price,stock_price,bond_rtn,stock_rtn,diff,bond_pos0.005@0,stock_pos,bond_pos0.015@0,stock_pos
2021-12-13 15:00:00,350.180,20.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2021-12-14 09:30:00,351.144,19.95,0.002753,-0.002500,0.005253,-1.0,1.0,0.0,0.0
2021-12-14 09:31:00,351.370,19.96,0.003398,-0.002000,0.005398,-1.0,1.0,0.0,0.0
2021-12-14 09:32:00,351.368,19.94,0.003393,-0.003000,0.006393,-1.0,1.0,0.0,0.0
2021-12-14 09:33:00,351.005,19.99,0.002356,-0.000500,0.002856,-1.0,1.0,0.0,0.0
2021-12-14 09:34:00,348.547,20.01,-0.004663,0.000500,-0.005163,1.0,-1.0,0.0,0.0
2021-12-14 09:35:00,348.900,19.97,-0.003655,-0.001500,-0.002155,1.0,-1.0,0.0,0.0
2021-12-14 09:36:00,350.196,20.07,0.000046,0.003500,-0.003454,1.0,-1.0,0.0,0.0
2021-12-14 09:37:00,350.226,20.08,0.000131,0.004000,-0.003869,1.0,-1.0,0.0,0.0
2021-12-14 09:38:00,351.280,20.02,0.003141,0.001000,0.002141,0.0,0.0,0.0,0.0


In [28]:
cnt = 0
for index,row in df_trade.iloc[1:,:].iterrows():
    if index.time() == time(15,0,0):
        cnt += 1
print(cnt)

34


In [29]:
len(rounds)

61